In [21]:
import os
from functools import partial
import json
import csv

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.metrics import efficiency_gap, mean_median, partisan_gini
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [2]:
graph_path = "./utah.json"
graph = Graph.from_json(graph_path)
with open (graph_path, "r") as myfile:
    data=myfile.readlines()
graph_dict = json.loads(data[0])

In [29]:
with open("test.csv", mode="w") as test:
    data_writer = csv.writer(test, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)   
    
    data_writer.writerow(["dem_votes", "rep_votes", "prec_pop", "node_num"])

    for n in graph.nodes:
        data_writer.writerow([graph.nodes[n]["SEN16D"],
                  graph.nodes[n]["SEN16R"],
                  graph.nodes[n]["TOTPOP"],
                  graph.nodes[n]["PrcncID"]])

In [3]:
pop_count = 0

for i in graph.nodes:
    pop_count += graph.nodes[i]["TOTPOP"]
    
print(pop_count)

2764056.0566752497


In [4]:
%%time
num_dist = 4

# Exercise: Compute exact population from your data.
pop = pop_count

my_updaters = {
    "population": updaters.Tally("TOTPOP", alias="population"),
    "cut_edges": cut_edges,
    "SEN16": Election("SEN16", {"democratic":"SEN16D","republican":"SEN16R"}),
    "PRES16": Election("SEN16", {"democratic":"PRES16D","republican":"PRES16R"}),
    "GOV16": Election("SEN16", {"democratic":"GOV16D","republican":"GOV16R"})
}


new_plan = recursive_tree_part(graph,
                               range(num_dist),
                               pop/num_dist,
                               "TOTPOP",
                               0.004,
                               1)
initial_partition = Partition(graph,
                              new_plan,
                              my_updaters)

CPU times: user 4.66 s, sys: 67.3 ms, total: 4.73 s
Wall time: 4.89 s


In [20]:
graph.nodes[0]

{'boundary_node': True,
 'boundary_perim': 2498.664338104136,
 'area': 751464858.6173844,
 'CountyID': 7,
 'VistaID': 'DU11',
 'PrcncID': 'DU11',
 'SbPrcnc': None,
 'AliasNm': None,
 'DsslvID': '7PDU11',
 'cnty_nm': 'Duchesne County',
 'cnty_fp': 49013,
 'jrsdctn': 'Duchesne',
 'PRES16D': 20.0,
 'PRES16R': 219,
 'PRES16I': 17,
 'SEN16D': 27,
 'SEN16R': 221,
 'GOV16D': 24,
 'GOV16R': 215,
 'TOTPOP': 504.4102544747859,
 'NH_WHITE': 430.29685596634425,
 'NH_BLACK': 0.004635374238234,
 'NH_AMIN': 1.01699533316232,
 'NH_ASIAN': 1.001545124746078,
 'NH_NHPI': 0.004635374238234,
 'NH_OTHER': 5.473073877000391,
 'NH_2MORE': 6.001544832824667,
 'HISP': 60.610968592231686,
 'H_WHITE': 18.991745359867213,
 'H_BLACK': 0.004635374238234,
 'H_AMIN': 3.003090249492156,
 'H_ASIAN': 0.003090249492156,
 'H_NHPI': 0.0,
 'H_OTHER': 36.25389455933765,
 'H_2MORE': 2.354512799804279,
 'VAP': 337.0797582972571,
 'HVAP': 33.244119822079185,
 'WVAP': 298.57512561836774,
 'BVAP': 0.004635374238234,
 'AMINVAP': 1

In [9]:
pop_prec = []
reps_prec = []
dems_prec = []

for n in tqdm_notebook(graph.nodes):
    pop_prec.append(graph.nodes[n]["TOTPOP"])
    reps_prec.append(graph.nodes[n]["SEN16R"])
    dems_prec.append(graph.nodes[n]["SEN16D"])

In [10]:
(dems_prec[0] - reps_prec[0]) / pop_prec[0]

-0.38460756552620307

In [79]:
ascending_DPC = []

for i in range(len(pop_prec)):
    if pop_prec[i] != 0:
        ascending_DPC.append([(dems_prec[i] - reps_prec[i]) / pop_prec[i], i])

In [80]:
len(ascending_DPC)

2119

In [81]:
ascending_DPC[:10]

[[-0.38460756552620307, 0],
 [-0.3130068205501431, 1],
 [-0.28875666363466074, 2],
 [-0.3016532241283324, 3],
 [-0.16906031750569048, 4],
 [-0.32026278007817355, 5],
 [-0.16872770425064787, 6],
 [-0.15076657045552386, 7],
 [-0.33661980586580514, 8],
 [-0.15659297915626635, 9]]

In [82]:
ascending_DPC.sort()

In [83]:
ascending_DPC[:10]

[[-33.47983173963706, 1362],
 [-23.7976727460307, 1829],
 [-20.22352546835022, 1670],
 [-14.031384817544831, 1252],
 [-10.502181943758941, 1313],
 [-6.687244415233582, 1311],
 [-6.660755182224414, 1169],
 [-6.371563798385555, 1351],
 [-5.0321267703814705, 1689],
 [-3.976267330082986, 713]]

In [85]:
descending_DPC = ascending_DPC.copy()
descending_DPC.reverse()

In [86]:
descending_DPC[:10]

[[13167.481966601481, 1629],
 [1.6727098944409131, 949],
 [0.5673002453763357, 944],
 [0.5224389262643541, 1879],
 [0.5056939270948492, 904],
 [0.4880164482201221, 923],
 [0.4592834882284874, 926],
 [0.45871102012966836, 910],
 [0.4438528834219532, 1893],
 [0.4045767740199276, 60]]

In [87]:
ascending_DPC[:10]

[[-33.47983173963706, 1362],
 [-23.7976727460307, 1829],
 [-20.22352546835022, 1670],
 [-14.031384817544831, 1252],
 [-10.502181943758941, 1313],
 [-6.687244415233582, 1311],
 [-6.660755182224414, 1169],
 [-6.371563798385555, 1351],
 [-5.0321267703814705, 1689],
 [-3.976267330082986, 713]]

In [36]:
I = pop_count/4
print(I)

691014.0141688124


In [111]:
pop_counter = 0
dem_qdist_1 = []
dem_qdist_2 = []

for j in descending_DPC:
    if pop_counter <= I:
        dem_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]]["TOTPOP"]
    elif pop_counter > I and pop_counter <= 2*I:
        dem_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]]["TOTPOP"]
    else:
        break

In [119]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in dem_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n]["SEN16D"]
    rep_votes_qdist_1 += graph.nodes[n]["SEN16R"]
    total_pop_qdist_1 += graph.nodes[n]["TOTPOP"]
    
for n in dem_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n]["SEN16D"]
    rep_votes_qdist_2 += graph.nodes[n]["SEN16R"]
    total_pop_qdist_2 += graph.nodes[n]["TOTPOP"]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.42558558047923367 0.6721111941795372
0.5510361257965515


In [120]:
pop_counter = 0
rep_qdist_1 = []
rep_qdist_2 = []

for j in ascending_DPC:
    if pop_counter <= I:
        rep_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]]["TOTPOP"]
    elif pop_counter > I and pop_counter <= 2*I:
        rep_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]]["TOTPOP"]
    else:
        break

In [121]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in rep_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n]["SEN16D"]
    rep_votes_qdist_1 += graph.nodes[n]["SEN16R"]
    total_pop_qdist_1 += graph.nodes[n]["TOTPOP"]
    
for n in rep_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n]["SEN16D"]
    rep_votes_qdist_2 += graph.nodes[n]["SEN16R"]
    total_pop_qdist_2 += graph.nodes[n]["TOTPOP"]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.8579890859102712 0.788031379787725
0.8279558133637631
